In [1]:
with open("prod.ini", "w") as prod, open("dev.ini", "w") as dev:
    prod.write("[Database]\n")
    prod.write("db_host=prod.mynetwork.com\n")
    prod.write("db_name=my_database\n")
    prod.write("\n[Server]\n")
    prod.write("port=8080\n")

    dev.write("[Database]\n")
    dev.write("db_host=dev.mynetwork.com\n")
    dev.write("db_name=my_database\n")
    dev.write("\n[Server]\n")
    dev.write("port=3000\n")


In [2]:
import configparser


class Config:
    def __init__(self, env="dev"):
        print(f"Loading config from {env} file")
        config = configparser.ConfigParser()
        file_name = f"{env}.ini"
        config.read(file_name)
        self.db_host = config["Database"]["db_host"]
        self.db_name = config["Database"]["db_name"]
        self.port = config["Server"]["port"]


In [3]:
config = Config("dev")

Loading config from dev file


In [4]:
config.__dict__

{'db_host': 'dev.mynetwork.com', 'db_name': 'my_database', 'port': '3000'}

In [5]:
help(config)  # no info about available attrs

Help on Config in module __main__ object:

class Config(builtins.object)
 |  Config(env='dev')
 |
 |  Methods defined here:
 |
 |  __init__(self, env='dev')
 |      Initialize self.  See help(type(self)) for accurate signature.
 |
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |
 |  __dict__
 |      dictionary for instance variables
 |
 |  __weakref__
 |      list of weak references to the object



In [6]:
import configparser


class Config:
    def __init__(self, env="dev"):
        print(f"Loading config from {env} file")
        config = configparser.ConfigParser()
        file_name = f"{env}.ini"
        config.read(file_name)

        for section_name in config.sections():
            for key, value in config[section_name].items():
                setattr(self, key, value)


In [7]:
c = Config("prod")
c.__dict__

Loading config from prod file


{'db_host': 'prod.mynetwork.com', 'db_name': 'my_database', 'port': '8080'}

In [8]:
class Section:
    def __init__(self, name, item_dict):
        """
        name: str
            name of the section
        item_dictL dict
            dictionary of named (key) config values (value)
        """
        self.name = name
        for key, value in item_dict.items():
            setattr(self, key, value)

In [9]:
class Config:
    def __init__(self, env="dev"):
        print(f"Loading config from {env} file")
        config = configparser.ConfigParser()
        file_name = f"{env}.ini"
        config.read(file_name)

        for section_name in config.sections():
            section = Section(section_name, config[section_name])
            setattr(self, section_name.casefold(), section)


In [10]:
config = Config()

Loading config from dev file


In [11]:
config.__dict__

{'database': <__main__.Section at 0x10d439010>,
 'server': <__main__.Section at 0x10d438fe0>}

In [12]:
help(Config)

Help on class Config in module __main__:

class Config(builtins.object)
 |  Config(env='dev')
 |
 |  Methods defined here:
 |
 |  __init__(self, env='dev')
 |      Initialize self.  See help(type(self)) for accurate signature.
 |
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |
 |  __dict__
 |      dictionary for instance variables
 |
 |  __weakref__
 |      list of weak references to the object



In [13]:
class SectionType(type):
    def __new__(cls, name, bases, cls_dict, section_name, item_dict):
        cls_dict["__doc__"] = f"Configs for {section_name} section"
        cls_dict["section_name"] = section_name
        for key, value in item_dict.items():
            cls_dict[key] = value
        return super().__new__(cls, name, bases, cls_dict)

In [14]:
class DatabaseSection(metaclass=SectionType, section_name="database", item_dict={"db_host": "db-host", "db_name": "test"}):
    pass


In [15]:
DatabaseSection.__dict__

mappingproxy({'__module__': '__main__',
              '__doc__': 'Configs for database section',
              'section_name': 'database',
              'db_host': 'db-host',
              'db_name': 'test',
              '__dict__': <attribute '__dict__' of 'DatabaseSection' objects>,
              '__weakref__': <attribute '__weakref__' of 'DatabaseSection' objects>})

In [16]:
help(DatabaseSection)  # now docs works as expected

Help on class DatabaseSection in module __main__:

class DatabaseSection(builtins.object)
 |  Configs for database section
 |
 |  Data descriptors defined here:
 |
 |  __dict__
 |      dictionary for instance variables
 |
 |  __weakref__
 |      list of weak references to the object
 |
 |  ----------------------------------------------------------------------
 |  Data and other attributes defined here:
 |
 |  db_host = 'db-host'
 |
 |  db_name = 'test'
 |
 |  section_name = 'database'



In [17]:
class ConfigType(type):

    def __new__(cls, name, bases, cls_dict, env):
        """
        env: str
            The env we are loading for the config (eg. prod, dev, stg)
        """
        cls_dict["__doc__"] = f"Config for {env}"
        cls_dict["env"] = env
        config = configparser.ConfigParser()
        file_name = f"{env}.ini"
        config.read(file_name)
        for section_name in config.sections():
            class_name = section_name.capitalize()
            class_attribute_name = section_name.casefold()
            section_items = config[section_name]
            bases = (object,)  # same as (,)
            section_cls_dict = {}
            Section = SectionType(
                class_name, 
                bases, 
                section_cls_dict, 
                section_name=section_name, 
                item_dict=section_items,
            )
            cls_dict[class_attribute_name] = Section
        return super().__new__(cls, name, bases, cls_dict)
        

In [18]:
class DevConfig(metaclass=ConfigType, env="dev"):
    """Dev config class"""


class ProdConfig(metaclass=ConfigType, env="prod"):
    """Prod config class"""


In [19]:
help(DevConfig)  # everything is filled in as expected

Help on class DevConfig in module __main__:

class DevConfig(builtins.object)
 |  Config for dev
 |
 |  Data descriptors defined here:
 |
 |  __dict__
 |      dictionary for instance variables
 |
 |  __weakref__
 |      list of weak references to the object
 |
 |  ----------------------------------------------------------------------
 |  Data and other attributes defined here:
 |
 |  database = <class '__main__.Database'>
 |      Configs for Database section
 |
 |
 |  env = 'dev'
 |
 |  server = <class '__main__.Server'>
 |      Configs for Server section



In [20]:
DevConfig.database.db_host  # autocomplition works!

'dev.mynetwork.com'

In [21]:
ProdConfig.__dict__

mappingproxy({'__module__': '__main__',
              '__doc__': 'Config for prod',
              'env': 'prod',
              'database': __main__.Database,
              'server': __main__.Server,
              '__dict__': <attribute '__dict__' of 'ProdConfig' objects>,
              '__weakref__': <attribute '__weakref__' of 'ProdConfig' objects>})

In [22]:
ProdConfig.server.port

'8080'

In [23]:
help(ProdConfig)

Help on class ProdConfig in module __main__:

class ProdConfig(builtins.object)
 |  Config for prod
 |
 |  Data descriptors defined here:
 |
 |  __dict__
 |      dictionary for instance variables
 |
 |  __weakref__
 |      list of weak references to the object
 |
 |  ----------------------------------------------------------------------
 |  Data and other attributes defined here:
 |
 |  database = <class '__main__.Database'>
 |      Configs for Database section
 |
 |
 |  env = 'prod'
 |
 |  server = <class '__main__.Server'>
 |      Configs for Server section



In [24]:
help(ProdConfig.database)

Help on class Database in module __main__:

class Database(builtins.object)
 |  Configs for Database section
 |
 |  Data descriptors defined here:
 |
 |  __dict__
 |      dictionary for instance variables
 |
 |  __weakref__
 |      list of weak references to the object
 |
 |  ----------------------------------------------------------------------
 |  Data and other attributes defined here:
 |
 |  db_host = 'prod.mynetwork.com'
 |
 |  db_name = 'my_database'
 |
 |  section_name = 'Database'

